# Blog maker 2000
I've always liked the vision of having my own tech blog, written from scratch. By scratch, I mean everything from the AWS tools to the web server to the stylesheets. All written by me, from scratch, at the command line. It's as much good practice for me as it is fun to do.

My first versions of this were static HTML files on S3. That was easy but involved copy and pasting from lots of HTML templates. Now I've moved my blog to EC2 where I can host everything via flask. To do this I needed to build some tools to manage content. Of course, there are tools that do this but I really wanted to stick to my original premise of making it from scratch. Plus this allows me to have exactly what I want and nothing else. 


### Design choices
* if there is no corresponding logic in the template to display an item, it will be ignored.
* all changes happen in place, not in copies
* currently there are no backups. backup and version control is managed via git.
* all changes happen locally and are not permanent until written in S3
* S3 goes straight to deployment


In [44]:
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import yaml
import time

myKeys = yaml.load(open('/home/billmanh/Downloads/Key_file', 'r'))
AWSSecretKey=myKeys['AWSSecretKey']
AWSAccessKeyId=myKeys['AWSAccessKeyId']
conn = S3Connection(AWSAccessKeyId, AWSSecretKey)
mybucket = conn.get_bucket('flaskgame')

#blog/d3_backend.json

In [45]:
#get a current article and edit it's content
myKey = mybucket.get_key("blog/d3_backend.json")

This is what a blog article looks like in the full template

In [46]:
article = yaml.load(myKey.get_contents_as_string())
article

{'author': 'William Harding',
 'content': [{'n': 'Formatting data for D3', 'type': 'h1'},
  {'n': 'Going from a CSV file to JSON', 'type': 'h1'},
  {'n': "This tutorial should give you a pretty good understanding of how to convert a data file into a format suited for a D3 visualization. In this example I'm using the data file that I got from my Personal Finance App and the Zoomable Circle Packing from mbostock's blocks (That guy is awesome!)",
   'type': 'p'},
  {'n': 'You should check out these other blogs in order because it covers the full range of a data deliverable from collection to presentation:',
   'type': 'p'},
  {'n': 'There are not enough tutorials on the boring backend stuff. Data Scientists spend an embarrassing amount of time getting data, cleaning it and processing it way before they ever get to make insights. Most tutorials contain cleaned data that was specially curated for that analysis or visualization.',
   'type': 'p'},
  {'n': 'First some easy functions that load

## Local functions to edit and create content

In [47]:
def createContent(aritcleID,title,author="William Harding",date=time.strftime("%d/%m/%Y"),keywords="data,vizualisation,technology"):
    article = {"articleID":aritcleID,
            "author":author,
            "date":date,
            "title":title,
            "content":[]}
    return article

def listArticles():
    for x in mybucket.list():
        if "blog" in x.name:
            print(x.name)
    return [a.name for a in mybucket.list() if "blog" in a.name]
    
#by default, content is placed at the end of the content array    
def addItemToContent(article,ctype,n):
    article['content'].append({"type":ctype,"n":n})
    return article

def addLinkToContent(article,link,t):
    article['content'].append({"type":"href","link":link,"t":t})
    return article
    
def addCodeToContent(article,n):
    article['content'].append({"type":"code","n":n})
    return article
    
def addDictToContent(article,d):
    article['content'].append(d)
    return article

#will overwrite if it already exists!
def addItemToMeta(article,dtype,n):
    article[dtype]=n
    return article

def addListToContent(article, listItems=["1","2","3"]):
    article['content'].append({"type":"list","l":listItems})
    return article

def changeItemContent(article,index,dtype,n):
    article['content'][index] = {"type":dtype,"n":n,"index":index}
    
def changeItemContent(article,index,dtype,n):
    article['content'][index] = {"type":dtype,"n":n,"index":index}
    
#enumerate content (to make sure that it stays in order)
def enumerateContent(article):
    counter = 1
    for i in article['content']:
        i["index"]=counter
        counter+=1
    return article
        
def publish(article):
    k = Key(mybucket)
    k.key = 'blog/' + article['articleID'] + '.json'
    k.set_contents_from_string(str(article))
    print(k.name)
    
def updateBlogListing(blogList=listArticles()):
    #QA step to remove items that are not blogs
    finalList = [a.replace("blog/","") for a in blogList if (len(a.replace("blog/",""))>1)&(".json" in a)]
    #add logic to control the display order
    

blog/
blog/article_1_test
blog/article_1_test.json
blog/d3_backend.json
blog/diy_cms.json


## This is how you make an article

In [48]:
article = createContent("diy_cms","Micro Blog Maker: DIY content managment")
article

{'articleID': 'diy_cms',
 'author': 'William Harding',
 'content': [],
 'date': '17/05/2017',
 'title': 'Micro Blog Maker: DIY content managment'}

In [49]:
#text for articles must be uploaded from 
paragraph = "I've always liked the vision of having my own tech blog, written from scratch. By scratch, I mean everything from the AWS tools to the web server to the stylesheets. All written by me, from scratch, at the command line. It's as much good practice for me as it is fun to do."
paragraph2 = "My first versions of this were static HTML files on S3. That was easy but involved copy and pasting from lots of HTML templates. Now I've moved my blog to EC2 where I can host everything via flask. To do this I needed to build some tools to manage content. Of course, there are tools that do this but I really wanted to stick to my original premise of making it from scratch. Plus this allows me to have exactly what I want and nothing else."
header = "design choices"

addItemToContent(article,"p",paragraph)
addItemToContent(article,"p",paragraph2)
addItemToContent(article,"h1",header)


{'articleID': 'diy_cms',
 'author': 'William Harding',
 'content': [{'n': "I've always liked the vision of having my own tech blog, written from scratch. By scratch, I mean everything from the AWS tools to the web server to the stylesheets. All written by me, from scratch, at the command line. It's as much good practice for me as it is fun to do.",
   'type': 'p'},
  {'n': "My first versions of this were static HTML files on S3. That was easy but involved copy and pasting from lots of HTML templates. Now I've moved my blog to EC2 where I can host everything via flask. To do this I needed to build some tools to manage content. Of course, there are tools that do this but I really wanted to stick to my original premise of making it from scratch. Plus this allows me to have exactly what I want and nothing else.",
   'type': 'p'},
  {'n': 'design choices', 'type': 'h1'}],
 'date': '17/05/2017',
 'title': 'Micro Blog Maker: DIY content managment'}

In [50]:
listItems = ["if there is no corresponding logic in the template to display an item, it will be ignored."
,"all changes happen in place, not in copies"
,"currently there are no backups. backup and version control is managed via git."
,"all changes happen locally and are not permanent until written in S3"
,"S3 goes straight to deployment"
]


addListToContent(article,listItems)


{'articleID': 'diy_cms',
 'author': 'William Harding',
 'content': [{'n': "I've always liked the vision of having my own tech blog, written from scratch. By scratch, I mean everything from the AWS tools to the web server to the stylesheets. All written by me, from scratch, at the command line. It's as much good practice for me as it is fun to do.",
   'type': 'p'},
  {'n': "My first versions of this were static HTML files on S3. That was easy but involved copy and pasting from lots of HTML templates. Now I've moved my blog to EC2 where I can host everything via flask. To do this I needed to build some tools to manage content. Of course, there are tools that do this but I really wanted to stick to my original premise of making it from scratch. Plus this allows me to have exactly what I want and nothing else.",
   'type': 'p'},
  {'n': 'design choices', 'type': 'h1'},
  {'l': ['if there is no corresponding logic in the template to display an item, it will be ignored.',
    'all changes h

In [51]:
addLinkToContent(article,"https://github.com/settings/profile","Check out the code to do this on my github page")

{'articleID': 'diy_cms',
 'author': 'William Harding',
 'content': [{'n': "I've always liked the vision of having my own tech blog, written from scratch. By scratch, I mean everything from the AWS tools to the web server to the stylesheets. All written by me, from scratch, at the command line. It's as much good practice for me as it is fun to do.",
   'type': 'p'},
  {'n': "My first versions of this were static HTML files on S3. That was easy but involved copy and pasting from lots of HTML templates. Now I've moved my blog to EC2 where I can host everything via flask. To do this I needed to build some tools to manage content. Of course, there are tools that do this but I really wanted to stick to my original premise of making it from scratch. Plus this allows me to have exactly what I want and nothing else.",
   'type': 'p'},
  {'n': 'design choices', 'type': 'h1'},
  {'l': ['if there is no corresponding logic in the template to display an item, it will be ignored.',
    'all changes h

## Send article to S3 so that it can be deployed

In [52]:
publish(article)

blog/diy_cms.json


### Posting the meta list so that articles show up in the main site



In [53]:
#not yet added, I'll come back to this once I have enough articles written.